In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF4
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61227 sha256=73959bd715a62246732c04e10ae01039c86e473f1f6cab8b7500e059cb767f1c
  Stored in directory: /root/.cache/pip/wheels/83/cc/14/cb307e5c99235c4497c7895cdb60b4f7ba2a738b6a5fc0d423
Successfully built PyPDF4


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


In [ ]:
#from PyPDF4 import PdfReader
import pdfplumber
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
files = [
    '/content/gdrive/My Drive/pdfs/qa.pdf',
    '/content/gdrive/My Drive/pdfs/trainn.pdf',
    '/content/gdrive/My Drive/pdfs/file.pdf',
    '/content/gdrive/My Drive/pdfs/fishfry-locations.pdf'
]

for file in files:
    reader = PdfReader(file)

In [ ]:
raw_text = '' ##empty string created
for i, page in enumerate(reader.pages): ##enumerate takes both index and corresponding value
    text = page.extract_text() ##extract text from each page
    if text:  ##ensuring only text is taken, images etc will be discarded
        raw_text += text ##string concatenation taking place

In [ ]:
raw_text[:]

'Description of Data:\nThe data is from the California Safe Cosmetics Program (CSCP) in the California Department of Public\nHealth. The primary purpose of the CSCP is to collect information on hazardous and potentially\nhazardous ingredients in cosmetic products sold in California and to make this information available to\nthe public.\nMany different chemicals are used in making cosmetics. Consumer and worker advocacy groups are\nconcerned because some cosmetic products contain chemicals known or suspected to cause cancer,\nbirth defects, or damage to the reproductive system. Those who work with cosmetics - including\nbarbers, hair stylists, and skin care, body care and nail salon workers - may be more vulnerable to the\nadverse health effects posed by these products because they handle greater quantities of cosmetics\nwith greater frequency.\nThe California Safe Cosmetics Act (“the Act”) was signed into law in 2005. For all cosmetic products sold\nin California, the Act requires the 

In [ ]:
text_splitter = CharacterTextSplitter(
    separator = "\n", #specifying character to split, newline here
    chunk_size = 1000, #maximum size of each chunk
    chunk_overlap  = 200, #overlap with adjacent chunks to avoid splitting in middle
    length_function = len,
)
texts = text_splitter.split_text(raw_text) ##splitting the extracted text according to parameters above

In [ ]:
len(texts)

3

In [ ]:
texts[:]

['Description of Data:\nThe data is from the California Safe Cosmetics Program (CSCP) in the California Department of Public\nHealth. The primary purpose of the CSCP is to collect information on hazardous and potentially\nhazardous ingredients in cosmetic products sold in California and to make this information available to\nthe public.\nMany different chemicals are used in making cosmetics. Consumer and worker advocacy groups are\nconcerned because some cosmetic products contain chemicals known or suspected to cause cancer,\nbirth defects, or damage to the reproductive system. Those who work with cosmetics - including\nbarbers, hair stylists, and skin care, body care and nail salon workers - may be more vulnerable to the\nadverse health effects posed by these products because they handle greater quantities of cosmetics\nwith greater frequency.\nThe California Safe Cosmetics Act (“the Act”) was signed into law in 2005. For all cosmetic products sold',
 'with greater frequency.\nThe Cal

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts[:], embeddings) ## document search index for input text and corresponding embeddings, FB AI SIMILARITY SEARCH

In [ ]:
docsearch

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = "Who sailed in the Nina - Pinta and Santa Maria"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."

In [ ]:
query = "What is the menu at Ancient Order of Hibernians"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The menu at Ancient Order of Hibernians includes haluska, mac and cheese, and shrimp. Dinners come with a choice of two sides (french fries, cole slaw, applesauce, or haluszki).'

In [ ]:
query = "When was The California Safe Cosmetics Act signed"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The California Safe Cosmetics Act was signed into law in 2005.'